In [7]:
import json

file_name = './data/sentiment/Books_small.json'

#take of the differec
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append((review['reviewText'], review['overall']))
        
reviews[5]

('Love the book, great story line, keeps you entertained.for a first novel from this author she did a great job,  Would definitely recommend!',
 4.0)